# My house modeling

## Description:

below the planimetry taken as a model, and divided into various layers through the .svg file. then i created with the function my solid model of the structure.

## Planimetry:

![Caption Figura](planimetry.jpg)

## Main Function: 

In [ ]:
from pyplasm import *
import csv

def ggpl_building_house():

	with open("perimetro.lines", "rb") as csvFile:
		reader = csv.reader(csvFile, delimiter=",")
		listPerimeter = []
		for row in reader:
			listPerimeter.append(POLYLINE([[float(row[0]), float(row[1])],[float(row[2]), float(row[3])]]))
	perimeter = STRUCT(listPerimeter)
	floor = SOLIDIFY(perimeter)
	xfactor = 15/SIZE([1])(perimeter)[0]
	yfactor = 15.1/SIZE([2])(perimeter)[0]
	perimeter = OFFSET([7,7])(perimeter)
	perimeter = PROD([perimeter, Q(3/xfactor)])

	with open("muri_interni.lines", "rb") as csvFile:
		reader = csv.reader(csvFile, delimiter=",")
		listInternalWalls = []
		for row in reader:
			listInternalWalls.append(POLYLINE([[float(row[0]), float(row[1])],[float(row[2]), float(row[3])]]))
	internalWalls = STRUCT(listInternalWalls)
	internalWalls = OFFSET([4,4])(internalWalls)


	internalWalls = PROD([internalWalls, Q(3/xfactor)])

	with open("porte.lines", "rb") as csvFile:
		reader = csv.reader(csvFile, delimiter=",")
		doorsList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):
				doorsList.append(MKPOL([cuboid,[[1,2,3,4]],None]))
				cuboid = []
				acc = 0
	doors = STRUCT(doorsList)
	doors = PROD([doors, Q(2.5/xfactor)])

	with open("finestre.lines", "rb") as csvFile:
		reader = csv.reader(csvFile, delimiter=",")
		windowList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):
				windowList.append(MKPOL([cuboid,[[1,2,3,4]],None]))
				cuboid = []
				acc = 0

	windows = STRUCT(windowList)
	windows = OFFSET([9,9])(windows)
	windows = PROD([windows, Q(SIZE([3])(perimeter)[0]/2.)])
	windows = T(3)(SIZE([3])(perimeter)[0]/4.)(windows)

	house = STRUCT([perimeter, internalWalls])
	house = DIFFERENCE([house, doors, windows])

	house = (S([1,2,3])([xfactor,yfactor, xfactor])(house))
	floor = (S([1,2,3])([xfactor,yfactor, xfactor])(floor))
	floor = TEXTURE("texture/parquetTexture.jpg")(floor)
	house = TEXTURE("texture/wallTexture.jpg")(house)

	VIEW(STRUCT([floor, house]))